In [14]:
import pandas as pd
import re
import numpy as np


from my_utils import analyze_dataframe

In [86]:
# Read the data
purchase_df = pd.read_csv('pwc_inventory/InvoicePurchases12312016.csv')
purchase_price_df = pd.read_csv('pwc_inventory/2017PurchasePricesDec.csv')
purchase_final_df = pd.read_csv('pwc_inventory/PurchasesFINAL12312016.csv')
inv_begin_df = pd.read_csv('pwc_inventory/BegInvFINAL12312016.csv')
inv_end_df = pd.read_csv('pwc_inventory/EndInvFINAL12312016.csv')
sales_df = pd.read_csv('pwc_inventory/SalesFINAL12312016.csv')

# Data Understanding

## dataset description

1. `InvoicePurchases12312016.csv` (Loaded into `purches`): This dataset records the company's procurement history up to December 31, 2016, detailing each purchase's item name, quantity, date, supplier, and cost.

2. `EndInvFINAL12312016.csv` (Loaded into `InvFinal`): This is a snapshot of the company's inventory at the close of December 31, 2016, listing item type and quantity.

3. `SalesFINAL12312016.csv` (Loaded into `sales`): This dataset presents all finalized sales transactions up to December 31, 2016, with details such as item sold, quantity, sale price, customer, and date.

4. `BegInvFINAL12312016.csv` (Loaded into `BegInv`): This is an overview of the company's inventory at the start of 2016, listing items and their corresponding quantities.

5. `2017PurchasePricesDec.csv` (Loaded into `PurchasePrice`): This dataset shows the purchase prices for inventory items for 2017 up to December, listing each item and its price.

6. `PurchasesFINAL12312016.csv` (Loaded into `PurchasesFinal`): This dataset contains finalized records of the company's inventory purchases up to December 31, 2016.



# Data Preprocessing

In [110]:
def is_float(n):
    try:
        float_n = float(n)
    except ValueError:
        return False
    else:
        return True

def preprocess_size(size):
    try:
        original_size = size  # Keep a copy of the original size for error reporting
        size = size.lower().strip()  # Convert to lowercase for uniform processing and remove spaces

        quantity = 1

        # Handle '750mL + 2/' format
        if '+' in size:
            size, _ = size.split('+')  # Only consider the main component
            size = size.strip()  # remove any potential spaces
        # Handle '50mL 4 Pk' format
        elif 'pk' in size:
            size, pack_info = size.split(' ', 1)  # Limit the number of splits to 1
            size = size.strip()  # remove any potential spaces
            quantity = int(pack_info.strip().split('pk')[0])  # remove any potential spaces
        # Convert gallons and ounces to mL
        elif 'gal' in size:
            size = re.sub(r'gal', '', size).strip()  # remove any potential spaces
            if is_float(size):
                size = float(size) * 3785.41  # Convert gallons to mL
                return size  # If we're here, we have a final result
            else:
                print(f"Size value is not numeric: '{original_size}'")
                return None
        elif 'oz' in size:
            size = re.sub(r'oz', '', size).strip()  # remove any potential spaces
            if is_float(size):
                size = float(size) * 29.5735  # Convert ounces to mL
                return size  # If we're here, we have a final result
            elif '/' in size:
                numerator, denominator = map(float, size.split('/'))  # Split the fraction and convert to float
                size = (numerator / denominator) * 29.5735  # Convert ounces to mL
                return size  # If we're here, we have a final result
            else:
                print(f"Size value is not numeric: '{original_size}'")
                return None

        # Handle '3/100mL' format
        elif '/' in size and 'pk' not in size:
            if 'oz' in size or 'ml' in size or 'l' in size:
                numerator, denominator = size.split('/')[0], re.split(r'[a-z]+', size.split('/')[1])[0]
                size = str(float(numerator) / float(denominator)) + "".join(re.findall(r'[a-z]+', size.split('/')[1]))
            else:
                quantity, size = size.split('/')
                quantity = int(quantity.strip())  # remove any potential spaces
                size = size.strip()  # remove any potential spaces

        # Convert sizes in liters to mL
        if 'ml' in size and is_float(re.sub(r'ml', '', size).strip()):
            size = re.sub(r'ml', '', size).strip()  # remove any potential spaces
            size = float(size) * quantity
        elif 'l' in size and is_float(re.sub(r'l', '', size).strip()):
            size = re.sub(r'l', '', size).strip()  # remove any potential spaces
            size = float(size) * 1000 * quantity
        elif size == 'liter':
            size = 1000.0
        elif is_float(size):  # If the size is just a floating point number, it's likely in mL
            size = float(size)
        else:
            print(f"Size value is not numeric: '{original_size}'")
            return None

        return size

    except Exception as e:
        print(f"Error processing size: '{original_size}'. Error: {e}")
        return None  # If any error occurs, return None


## Purchase Data

### Purchase

In [134]:
analyze_dataframe(purchase_df)

,Column,Data Type,Unique Count,Unique Sample,Missing Values,Missing Percentage
0,VendorNumber,int64,126,"[105, 4466, 388, 480, 516]",0,0.0
1,VendorName,object,129,"[ALTAMAR BRANDS LLC , AMERICAN VINTAGE...",0,0.0
2,InvoiceDate,object,373,"[2016-01-04, 2016-01-07, 2016-01-09, 2016-01-1...",0,0.0
3,PONumber,int64,5543,"[8124, 8137, 8169, 8106, 8170]",0,0.0
4,PODate,object,319,"[2015-12-21, 2015-12-22, 2015-12-24, 2015-12-2...",0,0.0
5,PayDate,object,382,"[2016-02-16, 2016-02-21, 2016-02-05, 2016-02-1...",0,0.0
6,Quantity,int64,2895,"[6, 15, 5, 10100, 1935]",0,0.0
7,Dollars,float64,5226,"[214.26, 140.55, 106.6, 137483.78, 15527.25]",0,0.0
8,Freight,float64,4052,"[3.47, 8.57, 4.61, 2935.2, 429.2]",0,0.0
9,Approval,object,2,"[None, Frank Delahunt]",0,0.0


In [135]:
def preprocess_purchase_df(purchase_df):
    # Make a copy of the dataframe
    purchase_df = purchase_df.copy()
    
    # Standardize text columns
    for col in ['VendorName', 'Approval']:
        purchase_df[col] = purchase_df[col].str.strip().str.lower()
    
    # Map Approval column to binary
    approval_map = {"none": 0, "frank delahunt": 1}
    purchase_df['Approval'] = purchase_df['Approval'].map(approval_map)
    
    return purchase_df


purchase_processed = preprocess_purchase_df(purchase_df)

### Purchase Price

In [138]:
analyze_dataframe(purchase_price_df)

,Column,Data Type,Unique Count,Unique Sample,Missing Values,Missing Percentage
0,Brand,int64,12261,"[58, 62, 63, 72, 75]",0,0.0
1,Description,object,11108,"[gekkeikan black & gold sake, herradura silver...",0,0.0
2,Price,float64,380,"[12.99, 36.99, 38.99, 34.99, 14.99]",0,0.0
3,Size,object,0,[None],12261,100.0
4,Volume,float64,33,"[750.0, 1000.0, 1750.0, 50.0, 375.0]",0,0.0
5,Classification,int64,2,"[1, 2]",0,0.0
6,PurchasePrice,float64,2314,"[9.28, 28.67, 30.46, 26.11, 10.94]",0,0.0
7,VendorNumber,int64,131,"[8320, 1128, 9165, 7245, 8112]",0,0.0
8,VendorName,object,133,"[shaw ross int l imp ltd, brown-forman corp, u...",0,0.0


In [116]:
def preprocess_price_df(df):
    # Create a copy to avoid changes to the original dataframe
    df = df.copy()
    
    # Fill missing values in 'Description' with 'unknown'
    df['Description'] = df['Description'].fillna('unknown')
    
    # Fill missing value in 'Volume' with -1 and convert the column to numeric
    df['Volume'] = pd.to_numeric(df['Volume'].fillna(-1), errors='coerce')
    
    # Fill missing values in 'Size' with 'unknown'
    df['Size'] = df['Size'].fillna('unknown')

    # If size is not 'unknown', standardize 'Size' to milliliters
    df['Size'] = df['Size'].apply(lambda x: preprocess_size(x) if x != 'Unknown' else -1)
    
    # Standardize text columns
    for col in ['Description', 'VendorName']:
        df[col] = df[col].str.strip().str.lower()

    return df

purchase_price_processed = preprocess_price_df(purchase_price_df)

In [137]:
analyze_dataframe(purchase_price_processed)

,Column,Data Type,Unique Count,Unique Sample,Missing Values,Missing Percentage
0,Brand,int64,12261,"[58, 62, 63, 72, 75]",0,0.0
1,Description,object,11108,"[gekkeikan black & gold sake, herradura silver...",0,0.0
2,Price,float64,380,"[12.99, 36.99, 38.99, 34.99, 14.99]",0,0.0
3,Size,int64,1,[-1],0,0.0
4,Volume,float64,33,"[750.0, 1000.0, 1750.0, 50.0, 375.0]",0,0.0
5,Classification,int64,2,"[1, 2]",0,0.0
6,PurchasePrice,float64,2314,"[9.28, 28.67, 30.46, 26.11, 10.94]",0,0.0
7,VendorNumber,int64,131,"[8320, 1128, 9165, 7245, 8112]",0,0.0
8,VendorName,object,133,"[shaw ross int l imp ltd, brown-forman corp, u...",0,0.0


In [115]:
def preprocess_purchase_final_df(purchase_final_df):
    # Create a copy to avoid changes to the original dataframe
    purchase_final_df = purchase_final_df.copy()
    
    # Fill missing values in 'Size' with 'unknown'
    purchase_final_df['Size'] = purchase_final_df['Size'].fillna('unknown')
    
    # Fill missing values in 'Size' with 'unknown'
    purchase_final_df['Size'] = purchase_final_df['Size'].fillna('unknown')

    # If size is not 'unknown', standardize 'Size' to milliliters
    purchase_final_df['Size'] = purchase_final_df['Size'].apply(lambda x: preprocess_size(x) if x != 'unknown' else -1)
    
    # Standardize text columns
    for col in ['Description', 'VendorName']:
        purchase_final_df[col] = purchase_final_df[col].str.strip().str.lower()

    # Convert date columns to datetime format
    for col in ['PODate', 'ReceivingDate', 'InvoiceDate', 'PayDate']:
        purchase_final_df[col] = pd.to_datetime(purchase_final_df[col])

    return purchase_final_df

purchase_final_processed = preprocess_purchase_final_df(purchase_final_df)

In [ ]:
purchase_df_processed, purchase_price_processed, purchase_final_processed

#### Standardize units in Size column

**Problem**: The 'Size' column contained diverse forms of measurements and various formats. For instance, sizes could appear as "750mL + 2/", "3/100mL", "50mL 4 Pk", "1 gal", or "5 oz".

**Solution**: 

Objective: To standardize size values into a uniform unit (milliliters), this was done in the following steps:

1. Converted the size strings to lrcase for uniform processing.
2. If the size included a '+' sign, only considered the main component and disregarded the rest.
3. For sizes formatted as "number/unit", split the number and the unit. Then convert 'oz' to mL by multiplying by 29.5735 and left 'ml' as is.
4. For sizes formatted as "number mL number Pk", split the string by the space. The first part became the size, and extracted the number of packs from the second part to calculate the total volume.
5. For sizes given in 'gal' and 'oz', convert them into mL using the appropriate conversion factors.
6. For sizes in 'l' and 'ml', strip away the units and kept only the numeric part, ensuring multiplied liters by 1000 to convert to milliliters.

Some examples of the transformations:

- "750mL + 2/" was standardized to 750.0
- "3/100mL" was standardized to 300.0
- "50mL 4 Pk" was standardized to 200.0
- "1 gal" was standardized to 3785.41
- "5 oz" was standardized to 147.8675



In [125]:
analyze_dataframe(inv_end_df)

,Column,Data Type,Unique Count,Unique Sample,Missing Values,Missing Percentage
0,InventoryId,object,224489,"[1_HARDERSFIELD_58, 1_HARDERSFIELD_62, 1_HARDE...",0,0.000
1,Store,int64,80,"[1, 2, 3, 4, 5]",0,0.000
2,City,object,67,"[HARDERSFIELD, ASHBORNE, HORNSEY, EANVERNESS, ...",1284,0.572
3,Brand,int64,9653,"[58, 62, 63, 72, 75]",0,0.000
4,Description,object,8732,"[Gekkeikan Black & Gold Sake, Herradura Silver...",0,0.000
5,Size,object,47,"[750mL, Liter, 750mL + 2/, 1.75L, 50mL]",0,0.000
6,onHand,int64,548,"[11, 7, 4, 18, 35]",0,0.000
7,Price,float64,354,"[12.99, 36.99, 38.99, 34.99, 14.99]",0,0.000
8,endDate,object,1,[2016-12-31],0,0.000


In [129]:
def preprocess_inventory_begin_df(inventory_df):
    # Create a copy to avoid changes to the original dataframe
    inventory_df = inventory_df.copy()
    
    # Standardize 'Size' to milliliters
    inventory_df['Size'] = inventory_df['Size'].apply(preprocess_size)
    
    # Standardize text columns
    for col in ['Description', 'City']:
        inventory_df[col] = inventory_df[col].str.strip()

    # Convert date columns to datetime format
    inventory_df['startDate'] = pd.to_datetime(inventory_df['startDate'])

    return inventory_df


def preprocess_inventory_end_df(inventory_df):
    # Create a copy to avoid changes to the original dataframe
    inventory_df = inventory_df.copy()
    
    # Standardize 'Size' to milliliters
    inventory_df['Size'] = inventory_df['Size'].apply(preprocess_size)
    
    # Standardize text columns
    for col in ['Description', 'City']:
        inventory_df[col] = inventory_df[col].str.strip()
    # Convert date columns to datetime format
    inventory_df['endDate'] = pd.to_datetime(inventory_df['endDate'])

    return inventory_df

inv_begin_processed = preprocess_inventory_begin_df(inv_begin_df)
inv_end_processed = preprocess_inventory_end_df(inv_end_df)

In [67]:
inv_begin_df['Size'] = inv_begin_df['Size'].apply(preprocess_size)
inv_end_df['Size'] = inv_end_df['Size'].apply(preprocess_size)

In [120]:
analyze_dataframe(sales_df)

,Column,Data Type,Unique Count,Unique Sample,Missing Values,Missing Percentage
0,InventoryId,object,170131,"[1_HARDERSFIELD_1004, 1_HARDERSFIELD_1005, 1_H...",0,0.0
1,Store,int64,79,"[1, 10, 11, 12, 13]",0,0.0
2,Brand,int64,7658,"[1004, 1005, 10058, 1006, 10062]",0,0.0
3,Description,object,6890,"[Jim Beam w/2 Rocks Glasses, Maker's Mark Comb...",0,0.0
4,Size,object,40,"[750mL, 375mL 2 Pk, 50mL 4 Pk, 750mL 3 Pk, 375...",0,0.0
5,SalesQuantity,int64,141,"[1, 2, 4, 3, 12]",0,0.0
6,SalesDollars,float64,3426,"[16.49, 32.98, 14.49, 69.98, 34.99]",0,0.0
7,SalesPrice,float64,273,"[16.49, 14.49, 34.99, 14.99, 3.99]",0,0.0
8,SalesDate,object,60,"[1/1/2016, 1/2/2016, 1/3/2016, 1/8/2016, 1/9/2...",0,0.0
9,Volume,int64,22,"[750, 375, 50, 200, 1500]",0,0.0


In [123]:
def preprocess_sales_df(sales_df):
    # Create a copy to avoid changes to the original dataframe
    sales_df = sales_df.copy()
    
    # Standardize 'Size' to milliliters
    sales_df['Size'] = sales_df['Size'].apply(preprocess_size)
    
    # Standardize text columns
    for col in ['Description', 'VendorName']:
        sales_df[col] = sales_df[col].str.strip().str.lower()

    # Convert date columns to datetime format
    sales_df['SalesDate'] = pd.to_datetime(sales_df['SalesDate'])

    return sales_df

sales_processed = preprocess_sales_df(sales_df)

In [132]:
#save to csv
purchase_df_processed.to_csv('data/2017PurchasePriceDec_processed.csv', index=False)
purchase_price_processed.to_csv('data/InvoicePurchases12312016_processed.csv', index=False)
purchase_final_processed.to_csv('data/PurchasesFINAL12312016_processed.csv', index=False)
inv_begin_processed.to_csv('data/BegInvFINAL12312016_processed.csv', index=False)
inv_end_processed.to_csv('data/EndInvFINAL12312016_processed.csv', index=False)
sales_processed.to_csv('data/SalesFINAL12312016_processed.csv', index=False)

In [133]:
purchase_df_processed

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,gekkeikan black & gold sake,12.99,-1,750.0,1,9.28,8320,shaw ross int l imp ltd
1,62,herradura silver tequila,36.99,-1,750.0,1,28.67,1128,brown-forman corp
2,63,herradura reposado tequila,38.99,-1,750.0,1,30.46,1128,brown-forman corp
3,72,no. 3 london dry gin,34.99,-1,750.0,1,26.11,9165,ultra beverage company llp
4,75,three olives tomato vodka,14.99,-1,750.0,1,10.94,7245,proximo spirits inc.
...,...,...,...,...,...,...,...,...,...
12256,44917,ferreira 10-yr tawny port,24.99,-1,750.0,2,16.55,90024,vinilandia usa
12257,44944,sanford santa rita pnt nr,22.99,-1,750.0,2,14.93,4425,martignetti companies
12258,45016,neal one lane bridg cab svgn,93.99,-1,750.0,2,61.43,10754,perfecta wines
12259,46011,folonari pnt nr venezie,12.99,-1,1500.0,2,8.90,9744,frederick wildman & sons
